In [13]:
# Export GCP service account key file to take care of authentication.
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/rajan/CREDENTIALS/rtc-ai-20240203-c915144c2c55.json"


In [38]:
import os
import time
from typing import List

import nbformat
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.tracers.stdout import elapsed
from langchain_google_vertexai import VertexAI, VertexAIEmbeddings
import vertexai
import requests
from langchain.schema.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language

In [2]:
!pip3 install --upgrade --user -q google-cloud-aiplatform \
                                langchain \
                                langchain_google_vertexai \
                                langchain-community \
                                faiss-cpu \
                                nbformat

In [4]:
# Initialize vertex AI
import vertexai
PROJECT_ID = "rtc-ai-20240203"      # Replace this with your GCP project id.
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

print(f"Vertex AI API version: {vertexai.__version__}")


Vertex AI API version: 1.65.0


In [7]:
# Create Code LLM
code_llm = VertexAI(
    model_name="gemini-1.5-pro",
    max_output_tokens=2048,
    temperature=0.1,
    verbose=False,
)


In [19]:
# Crawl through git repo and return list of all ipynb files in the repo.
def crawl_git_repo(url: str, is_sub_dir: bool, access_token: str):

    if is_sub_dir:
        api_url = url
    else:
        api_url = f"https://api.github.com/repos/{url}/contents"

    header = {
        "Accept": "application/vnd.github.v3+json",
        "Authorization": f"token {access_token}",
    }
    response = requests.get(api_url, headers=header)

    # Check for any request errors
    response.raise_for_status()

    files = []
    contents = response.json()


    for item in contents:
        if (item["type"] == "file"
            and (item["name"].endswith(".py") or item["name"].endswith(".ipynb"))
        ):
            files.append(item["html_url"])
        elif item["type"] == "dir" and not item["name"].startswith("."):
            sub_files = crawl_git_repo(item["url"], True, access_token)
            time.sleep(0.1)         # Wait for 100 milliseconds before next github api call.
            files.extend(sub_files)

    return files

In [20]:
print(GITHUB_TOKEN)

In [21]:
REPO_URL = "ibm-et/jupyter-samples"
code_files_urls = crawl_git_repo(REPO_URL, False, GITHUB_TOKEN)

# Write code file urls list to a file so that we don't have to download each time.
with open("code_files_urls.txt", "w") as f:
    for url in code_files_urls:
        f.write(url + "\n")

print("Done")

Done


In [23]:
code_files_urls[0:10]

['https://github.com/ibm-et/jupyter-samples/blob/master/airline/Exploration%20of%20Airline%20On-Time%20Performance.ipynb',
 'https://github.com/ibm-et/jupyter-samples/blob/master/bluemix-spark-cloudant/1-Streaming-Meetups-to-IBM-Cloudant-using-Spark.ipynb',
 'https://github.com/ibm-et/jupyter-samples/blob/master/bluemix-spark-cloudant/2-Reading-Meetups-from-IBM-Cloudant-using-Spark.ipynb',
 'https://github.com/ibm-et/jupyter-samples/blob/master/elasticity/Elasticity%20Experiment.ipynb',
 'https://github.com/ibm-et/jupyter-samples/blob/master/hacks/IPython%20Parallel%20and%20R.ipynb',
 'https://github.com/ibm-et/jupyter-samples/blob/master/hacks/Webserver%20in%20a%20Notebook.ipynb',
 'https://github.com/ibm-et/jupyter-samples/blob/master/hacks/instaquery.ipynb',
 'https://github.com/ibm-et/jupyter-samples/blob/master/hn/Hacker%20News%20Runner.ipynb',
 'https://github.com/ibm-et/jupyter-samples/blob/master/hn/Hacker%20News%20and%20AlchemyAPI.ipynb',
 'https://github.com/ibm-et/jupyter-sa

# File: my-code-rag-2.py

In [25]:
##########################################
# Extract code from the Jupyter notebook
##########################################
# Extracts the python code from an ipynb file from github
def extract_python_code_from_ipynb(github_url, cell_type="code"):
    raw_url = github_url.replace("github.com", "raw.githubusercontent.com").replace(
        "/blob/", "/"
    )

    response = requests.get(raw_url)
    response.raise_for_status()  # Check for any request errors

    notebook_content = response.text

    notebook = nbformat.reads(notebook_content, as_version=nbformat.NO_CONVERT)

    python_code = None

    if hasattr(notebook, 'worksheets'):
        cells = notebook.worksheets[0].cells
    else:
        cells = notebook.cells

    for cell in cells:
        if cell.cell_type == cell_type:

            if hasattr(cell, 'input'):
                code = cell.input
            else:
                code = cell.source

            if not python_code:
                python_code = code
            else:
                python_code += "\n" + code

    return python_code

In [26]:
# Read the file urls from text file.
with open("code_files_urls.txt") as f:
    code_files_urls = f.read().splitlines()
print(f"Total file count is {len(code_files_urls)}")

Total file count is 23


In [31]:
code_strings = []

for i in range(0, len(code_files_urls)):
    if code_files_urls[i].endswith("ipynb"):
        content = extract_python_code_from_ipynb(code_files_urls[i], "code")

        if content is not None:
            doc = Document(
                page_content=content,
                metadata={
                    "url": code_files_urls[i],
                    "file_index": i
                }
            )

            code_strings.append(doc)

In [33]:
code_strings[0]

Document(metadata={'url': 'https://github.com/ibm-et/jupyter-samples/blob/master/airline/Exploration%20of%20Airline%20On-Time%20Performance.ipynb', 'file_index': 0}, page_content='!pip install cloudant\nimport cloudant\naccount = cloudant.Account(\'parente\')\ndatabase = account.database(\'rita_transtats_2014_06\')\ndatabase.get().json()\nitems = []\nfor i, item in enumerate(database.all_docs(params={\'include_docs\' : True})):\n    if i > 1: break\n    items.append(item)\nprint items\nimport pandas\npandas.DataFrame([item[\'doc\'] for item in items])\ncolumns = [u\'FL_DATE\', u\'ORIGIN_STATE_ABR\', u\'DEST_STATE_ABR\', u\'ARR_DEL15\', u\'ARR_DELAY_NEW\', u\'DEP_DEL15\', u\'DEP_DELAY_NEW\', u\'DISTANCE\', u\'DISTANCE_GROUP\',]\n%%time\ndfs = []\nbuff = []\nfor i, item in enumerate(database.all_docs(params={\'include_docs\' : True})):\n    buff.append(item[\'doc\'])\n    if i > 0 and i % 20000 == 0:\n        print \'Processed #{}\'.format(i)\n        df = pandas.DataFrame(buff, columns=

In [36]:
# Chunk code strings
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=2000,
    chunk_overlap=200
)

texts = text_splitter.split_documents(code_strings)
print(len(texts))


# Rate Limiter function
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield       # Request making happens here
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)

# Initialize Embeddings API
class CustomVertexAIEmbeddings(VertexAIEmbeddings):
    requests_per_minute: int
    num_instances_per_batch: int
    model_name: str
    
    def embed_documents(
        self, texts: List[str], batch_size: int = 0
    ) -> List[List[float]]:
        
        # setup rate limiter
        limiter = rate_limit(self.requests_per_minute)
        
        results = []
        
        docs = list(texts)
        
        while docs:
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch : ]
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)
        
        return [r.values for r in results]



embeddings = CustomVertexAIEmbeddings(
    requests_per_minute = 100,
    num_instances_per_batch = 5,
    model_name = "textembedding-gecko@latest"
)

# Create Index from embedded code chunks
db = FAISS.from_documents(texts, embeddings)

# Initialize the retriever
retriever = db.as_retriever(
    search_type="similarity",       # Other option is "mmr"
    search_kwargs={"k": 5}
)


120
Waiting
...

# Lets ask a question using RAG approach

In [39]:
# RAG template
prompt_RAG = """
    You are a proficient python developer. Respond with the syntactically correct code for to the question below. Make sure you follow these rules:
    1. Use context to understand the APIs and how to use it & apply.
    2. Do not add license information to the output code.
    3. Do not include Colab code in the output.
    4. Ensure all the requirements in the question are met.

    Question:
    {question}

    Context:
    {context}

    Helpful Response :
    """

prompt_RAG_template = PromptTemplate(
    template=prompt_RAG, input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_llm(
    llm=code_llm,
    prompt=prompt_RAG_template,
    retriever=retriever,
    return_source_documents=True
)

In [40]:
user_question = "Create a Python function that takes a prompt and predicts using langchain.llms interface with Vertex AI text-bison model"
results = qa_chain.invoke(input={"query": user_question})
print(results["result"])

```python
from langchain.llms import VertexAI
from typing import Any

def predict_with_vertex_ai(prompt: str) -> Any:
    """Takes a prompt and predicts using langchain.llms interface with Vertex AI text-bison model.

    Args:
        prompt: The input prompt for the model.

    Returns:
        The response from the Vertex AI text-bison model.
    """
    llm = VertexAI(model_name="text-bison@001")
    return llm(prompt)
```


# Generate python code using LLM (without RAG)

In [41]:
prompt_zero_shot = """
    You are a proficient python developer. Respond with the syntactically correct & concise code for to the question below.

    Question:
    {question}

    Output Code :
    """

In [42]:
response = code_llm.invoke(input=user_question, max_output_tokens=2048, temperature=0.1)
print(response)

```python
from langchain.llms import VertexAI
from typing import Optional

def predict_with_vertex_ai(prompt: str, 
                           model_name: Optional[str] = "text-bison@001", 
                           temperature: Optional[float] = 0.7, 
                           max_output_tokens: Optional[int] = 256) -> str:
  """
  Predicts text using a Vertex AI text generation model.

  Args:
      prompt: The input prompt for text generation.
      model_name: The name of the Vertex AI text generation model to use. 
                  Defaults to "text-bison@001".
      temperature: Controls the randomness of the generated text. Higher values 
                   result in more random text. Defaults to 0.7.
      max_output_tokens: The maximum number of tokens to generate in the output. 
                         Defaults to 256.

  Returns:
      The generated text.
  """

  llm = VertexAI(model_name=model_name, 
                 temperature=temperature, 
                 max_outpu